# Set up modular

In [ ]:
#@title Create classification_ai folder
from pathlib import Path

CLASS_PATH=Path("classification_ai/")
CLASS_PATH.mkdir(parents=True,exist_ok=True)


In [ ]:
%%writefile classification_ai/download_and_decompress_data.py
#@title Download data
import os
import requests
import zipfile
import tarfile
import gzip
from concurrent.futures import ThreadPoolExecutor

def download_file(url, folder='.'):
    file_name = os.path.join(folder, url.split('/')[-1])

    if os.path.exists(file_name):
        return file_name  # Return file path if it already exists

    print(f"Downloading {url}")
    response = requests.get(url, stream=True)

    with open(file_name, 'wb') as file:
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:
                file.write(chunk)

    print(f"Download complete: {file_name}")
    return file_name

def decompress_file(file_name):
    if file_name.endswith('.zip'):
        with zipfile.ZipFile(file_name, 'r') as zip_ref:
            zip_ref.extractall(os.path.dirname(file_name))
        os.remove(file_name)
    elif file_name.endswith('.tar.gz') or file_name.endswith('.tgz'):
        with tarfile.open(file_name, 'r:gz') as tar_ref:
            tar_ref.extractall(os.path.dirname(file_name))
        os.remove(file_name)
    elif file_name.endswith('.gz'):
        with gzip.open(file_name, 'rb') as gz_ref:
            with open(file_name[:-3], 'wb') as uncompressed_file:
                uncompressed_file.write(gz_ref.read())
        os.remove(file_name)

def is_already_downloaded_or_decompressed(url, folder):
    file_name = os.path.join(folder, url.split('/')[-1])
    decompressed_file_name = file_name

    if file_name.endswith('.zip'):
        decompressed_file_name = file_name[:-4]
    elif file_name.endswith('.tar.gz') or file_name.endswith('.tgz'):
        decompressed_file_name = file_name[:-7]
    elif file_name.endswith('.gz'):
        decompressed_file_name = file_name[:-3]

    return os.path.exists(file_name) or os.path.exists(decompressed_file_name)

def download_and_decompress(url, folder='.'):
    if not os.path.exists(folder):
        os.makedirs(folder)

    if is_already_downloaded_or_decompressed(url, folder):
        print(f"{url} was already downloaded and decompressed in {folder}.")
        return

    with ThreadPoolExecutor(max_workers=2) as executor:
        file_name = executor.submit(download_file, url, folder).result()

        if file_name:
            print(f"Downloaded {file_name} successfully.")
            executor.submit(decompress_file, file_name)
            print(f"Decompressed {file_name}.")
        else:
            print(f"Failed to download {url}.")

#download_and_decompress("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip","data")




Writing classification_ai/download_and_decompress_data.py


In [ ]:
%%writefile classification_ai/data_setup.py
#@title Data set up

"""
Contains functionality for creating PyTorch DataLoaders for
image classification data.
"""
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
NUM_WORKERS = os.cpu_count()

def create_dataloaders(
    data: str,
    transform: transforms.Compose,
    batch_size: int,
    train_test_ratio: float,
    num_workers: int=NUM_WORKERS,
):
  """Creates DataLoaders.

  Takes in a training directory and testing directory path and turns
  them into PyTorch Datasets and then into PyTorch DataLoaders.

  Args:
    data: path to directory of data (food-101)
    transform: torchvision transforms to perform on training and testing data.
    batch_size: Number of samples per batch in each of the DataLoaders.
    train_test_ratio: Percentage of data you want to be TEST data.
    num_workers: An integer for number of workers per DataLoader.

  Returns:
    A dataloader.
    Where class_names is a list of the target classes.
    Example usage:
      train_dataloader, test_dataloader, class_names = \
        = create_dataloaders(data=path/to/food-101,
                             transform=some_transform,
                             batch_size=32,
                             num_workers=os.cpu_count())
  """
  # Use ImageFolder to create dataset(s)
  data = datasets.ImageFolder(data, transform=transform)
  train_data, test_data = random_split(data,[1-train_test_ratio, train_test_ratio])
  # Get class names
  class_names = data.classes

  # Turn images into data loaders
  train_dataloader = DataLoader(
      train_data,
      batch_size=batch_size,
      shuffle=True,
      num_workers=num_workers,
      pin_memory=True,
  )
  test_dataloader = DataLoader(
      test_data,
      batch_size=batch_size,
      shuffle=False, # don't need to shuffle test data
      num_workers=num_workers,
      pin_memory=True,
  )

  return train_dataloader, test_dataloader, class_names


Writing classification_ai/data_setup.py


In [ ]:
%%writefile classification_ai/build_VGG_model.py
#@title Build Model
"""
Contains PyTorch model code to instantiate a TinyVGG model.
"""
import torch
from torch import nn

class TinyVGG(nn.Module):
  """Creates the TinyVGG architecture.

  Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
  See the original architecture here: https://poloclub.github.io/cnn-explainer/

  Args:
    input_shape: An integer indicating number of input channels.
    hidden_units: An integer indicating number of hidden units between layers.
    output_shape: An integer indicating number of output units.
  """
  def __init__(self, input_shape: int, size_image: int, hidden_units: int, output_shape: int) -> None:
      super().__init__()
      self.conv_block_1 = nn.Sequential(
          nn.Conv2d(in_channels=input_shape,
                    out_channels=hidden_units,
                    kernel_size=5,
                    stride=1,
                    padding=2),
          nn.ReLU(),
          nn.Conv2d(in_channels=hidden_units,
                    out_channels=hidden_units,
                    kernel_size=5,
                    stride=1,
                    padding=2),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,
                        stride=2)
      )
      self.conv_block_2 = nn.Sequential(
          nn.Conv2d(hidden_units, hidden_units, kernel_size=5, padding=2),
          nn.ReLU(),
          nn.Conv2d(hidden_units, hidden_units, kernel_size=5, padding=2),
          nn.ReLU(),
          nn.MaxPool2d(2)
      )
      self.classifier = nn.Sequential(
          nn.Flatten(),
          # Where did this in_features shape come from?
          # It's because each layer of our network compresses and changes the shape of our inputs data.
          nn.Linear(in_features=hidden_units*int(size_image/4)**2,
                    out_features=output_shape)
      )
  def forward(self, x: torch.Tensor):
      return self.classifier(self.conv_block_2(self.conv_block_1(x)))
      # return self.classifier(self.conv_block_2(self.conv_block_1(x))) # <- leverage the benefits of operator fusion


Writing classification_ai/build_VGG_model.py


In [ ]:
%%writefile classification_ai/engine.py
#@title Model Education
"""
Contains functions for training and testing a PyTorch model.
"""
import torch

from tqdm.auto import tqdm
from typing import Dict, List, Tuple
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
    """Trains a PyTorch model for a single epoch.
    Turns a target PyTorch model to training mode and then
    runs through all of the required training steps (forward
    pass, loss calculation, optimizer step).
    Args:
    model: A PyTorch model to be trained.
    dataloader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    device: A target device to compute on (e.g. "cuda" or "cpu").
    Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_accuracy). For example:
    (0.1112, 0.8743)
    """
    # Put model in train mode
    model.train()

    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0

    # Loop through data loader data batches
    for batch, (X, y) in enumerate(dataloader):
        # Send data to target device
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate  and accumulate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    # Adjust metrics to get average loss and accuracy per batch
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
    """Tests a PyTorch model for a single epoch.
    Turns a target PyTorch model to "eval" mode and then performs
    a forward pass on a testing dataset.
    Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").
    Returns:
    A tuple of testing loss and testing accuracy metrics.
    In the form (test_loss, test_accuracy). For example:
    (0.0223, 0.8985)
    """
    # Put model in eval mode
    model.eval()

    # Setup test loss and test accuracy values
    test_loss, test_acc = 0, 0

    # Turn on inference context manager
    with torch.inference_mode():
        # Loop through DataLoader batches
        for batch, (X, y) in enumerate(dataloader):
            # Send data to target device
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred_logits = model(X)

            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            # Calculate and accumulate accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

    # Adjust metrics to get average loss and accuracy per batch
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
    """Trains and tests a PyTorch model.
    Passes a target PyTorch models through train_step() and test_step()
    functions for a number of epochs, training and testing the model
    in the same epoch loop.
    Calculates, prints and stores evaluation metrics throughout.
    Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").
    Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for
    each epoch.
    In the form: {train_loss: [...],
              train_acc: [...],
              test_loss: [...],
              test_acc: [...]}
    For example if training for epochs=2:
             {train_loss: [2.0616, 1.0537],
              train_acc: [0.3945, 0.3945],
              test_loss: [1.2641, 1.5706],
              test_acc: [0.3400, 0.2973]}
    """
    # Create empty results dictionary
    results = {"train_loss": [],
               "train_acc": [],
               "test_loss": [],
               "test_acc": []
    }

    # Loop through training and testing steps for a number of epochs
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
        test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

        # Print out what's happening
        print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

    # Return the filled results at the end of the epochs
    return results




Writing classification_ai/engine.py


In [ ]:
%%writefile classification_ai/utils.py
#@title Save Model
"""
Contains various utility functions for PyTorch model training and saving.
"""
import torch
from pathlib import Path

def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):
    """Saves a PyTorch model to a target directory.
    Args:
    model: A target PyTorch model to save.
    target_dir: A directory for saving the model to.
    model_name: A filename for the saved model. Should include
      either ".pth" or ".pt" as the file extension.
    Example usage:
    save_model(model=model_0,
               target_dir="models",
               model_name="05_going_modular_tingvgg_model.pth")
    """
    # Create target directory
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True,
                        exist_ok=True)

    # Create model save path
    assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
    model_save_path = target_dir_path / model_name

    # Save the model state_dict()
    print(f"[INFO] Saving model to: {model_save_path}")
    torch.save(obj=model.state_dict(),
             f=model_save_path)

Writing classification_ai/utils.py


In [ ]:
%%writefile train.py
#@title Train
"""
Trains a PyTorch image classification model using device-agnostic code.
"""

import os
import argparse

import torch

from torchvision import transforms

from classification_ai import data_setup, engine, build_VGG_model, utils

# Create a parser
parser = argparse.ArgumentParser(description="Get some hyperparameters.")

# Get an arg for num_epochs
parser.add_argument("--num_epochs",
                     default=10,
                     type=int,
                     help="the number of epochs to train for")

# Get an arg for batch_size
parser.add_argument("--batch_size",
                    default=3,
                    type=int,
                    help="number of samples per batch")

# Get an arg for hidden_units
parser.add_argument("--hidden_units",
                    default=10,
                    type=int,
                    help="number of hidden units in hidden layers")

# Get an arg for learning_rate
parser.add_argument("--learning_rate",
                    default=0.001,
                    type=float,
                    help="learning rate to use for model")

# Create an arg for training directory
parser.add_argument("--data",
                    default="data",
                    type=str,
                    help="directory file path to data in standard image classification format")

# Get our arguments from the parser
args = parser.parse_args()

# Setup hyperparameters
NUM_EPOCHS = args.num_epochs
BATCH_SIZE = args.batch_size
HIDDEN_UNITS = args.hidden_units
LEARNING_RATE = args.learning_rate
print(f"[INFO] Training a model for {NUM_EPOCHS} epochs with batch size {BATCH_SIZE} using {HIDDEN_UNITS} hidden units and a learning rate of {LEARNING_RATE}")

# Setup directories
data = args.data
print(f"[INFO] Data file: {data}")

# Setup target device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create transforms
data_transform = transforms.Compose([
  transforms.Resize((64, 64)),
  transforms.ToTensor()
])

# Create DataLoaders with help from data_setup.py
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(
    data=data,
    transform=data_transform,
    batch_size=BATCH_SIZE,
    train_test_ratio=0.2
)

# Create model with help from build_VGG_model.py
model = build_VGG_model.TinyVGG(
    input_shape=3,
    hidden_units=HIDDEN_UNITS,
    output_shape=len(class_names),
    size_image=64
).to(device)

# Set loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=LEARNING_RATE)

# Start training with help from engine.py
engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device)

# Save the model with help from utils.py
utils.save_model(model=model,
                 target_dir="models",
                 model_name="05_going_modular_script_mode_tinyvgg_model.pth")

Writing train.py


## Deploiment

In [ ]:
#@title Donwload data
SIZE_IMAGE=224
from classification_ai.download_and_decompress_data import  download_and_decompress
from classification_ai.data_setup import create_dataloaders
from torchvision import transforms #transformers important
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from classification_ai import build_VGG_model

download_and_decompress("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip","data")

#transform
data_transform = transforms.Compose([
    # This is just for optimization
    transforms.TrivialAugmentWide(num_magnitude_bins=31),
    #Resize our images to 64*64
    transforms.Resize(size=(SIZE_IMAGE,SIZE_IMAGE)),
    #Flip the images randomly on the horizontal
    transforms.RandomHorizontalFlip(p=0.5),
    #Turn the image into a torch.Tensor
    transforms.ToTensor(),
    #RandomErasing-> Randomily errase some pixels of a image!
    transforms.RandomErasing(p=0.15)
])
#Set up dataloaders
#this is more or less, since i changed create_dataloaders to something else..

"""
Contains functionality for creating PyTorch DataLoaders for
image classification data.
"""

NUM_WORKERS = os.cpu_count()
#Bugged in Colab!

train_dataloader, test_dataloader, class_names = create_dataloaders(data=r"data/train",transform=data_transform,batch_size=4,train_test_ratio=0.2)


# Import model_builder.py
device = "cuda" if torch.cuda.is_available() else "cpu"

# Instantiate an instance of the model from the "model_builder.py" script
torch.manual_seed(42)
model_1 = build_VGG_model.TinyVGG(input_shape=3,
                              hidden_units=30,
                              output_shape=len(class_names),size_image=SIZE_IMAGE).to(device)

model_1

Downloaded data/pizza_steak_sushi.zip successfully.
Decompressed data/pizza_steak_sushi.zip.


TinyVGG(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 30, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): Conv2d(30, 30, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(30, 30, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): Conv2d(30, 30, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=94080, out_features=3, bias=True)
  )
)

# Train

In [ ]:
#@title Train Model
!python train.py --hidden_units 200 --batch_size 6 --learning_rate 0.001 --num_epochs 10


[INFO] Training a model for 10 epochs with batch size 6 using 200 hidden units and a learning rate of 0.001
[INFO] Data file: data
  0% 0/10 [00:00<?, ?it/s]Epoch: 1 | train_loss: 0.7143 | train_acc: 0.7458 | test_loss: 0.5814 | test_acc: 0.7333
 10% 1/10 [00:03<00:31,  3.45s/it]Epoch: 2 | train_loss: 0.5961 | train_acc: 0.7542 | test_loss: 0.5925 | test_acc: 0.7333
 20% 2/10 [00:06<00:25,  3.15s/it]Epoch: 3 | train_loss: 0.5581 | train_acc: 0.7542 | test_loss: 0.6306 | test_acc: 0.7333
 30% 3/10 [00:09<00:21,  3.06s/it]Epoch: 4 | train_loss: 0.5956 | train_acc: 0.7542 | test_loss: 0.6010 | test_acc: 0.7333
 40% 4/10 [00:12<00:18,  3.08s/it]Epoch: 5 | train_loss: 0.5626 | train_acc: 0.7542 | test_loss: 0.6126 | test_acc: 0.7333
 50% 5/10 [00:15<00:15,  3.03s/it]Epoch: 6 | train_loss: 0.5707 | train_acc: 0.7542 | test_loss: 0.5865 | test_acc: 0.7333
 60% 6/10 [00:18<00:11,  3.00s/it]Epoch: 7 | train_loss: 0.5661 | train_acc: 0.7542 | test_loss: 0.5801 | test_acc: 0.7333
 70% 7/10 [00:21